In [2]:
#system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns


import pickle

In [3]:

import biosppy
from biosppy import storage
from biosppy.signals import ecg


# misc
import warnings

#signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

time_zone = 'Europe/Berlin'
tz = pytz.timezone(time_zone)

## Read and manipulate Psychopy dataframe

In [4]:
def read_manipulate_psychopy(psychopy_path):
    
    psychopy_df = pd.read_csv(psychopy_path)
    col_list = [col for col in psychopy_df.columns if col.endswith('_ts')]
    #col_list.insert(0, "Reference_time")
    psychopy_df_selected = psychopy_df[col_list]
    
    return psychopy_df_selected


def get_nonNan_list_psychopy(psychopy_df, col_name):
    
    
    selected_vals=[val for val in psychopy_df[col_name].to_list() if not(math.isnan(val))]
    
    return selected_vals


def sel_pre_analysis_file(events_path_list):
    
    for sel_event_path in events_path_list:
        fol_path, file_name_ii=os.path.split(sel_event_path)
        if "trigger" in file_name_ii:
            events_with_trigger= sel_event_path
            
            
    return sel_event_path




    

# Handling delays in the file 

In [5]:
def from_excel_val_to_timestamp(ts_val):
    

    
    ##########
    ts_val = str(ts_val)
    ts_val = str(ts_val.replace('.',''))
    ts_val= ts_val[0:10] + '.' + ts_val[10:]
    ts_val = float(ts_val)
    return ts_val

def rreplace(string: str, find: str, replace: str, n_occurences: int) -> str:
    """
    Given a `string`, `find` and `replace` the first `n_occurences`
    found from the right of the string.
    """
    temp = string.rsplit(find, n_occurences)
    return replace.join(temp)

def get_offset_timestamp(timestamp, offset_mins):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time = dt.datetime.fromtimestamp(timestamp, tz)
    time_change = dt.timedelta(minutes=offset_mins)
    new_time = local_time + time_change
    new_timestamp =  dt.datetime.timestamp(new_time)
    return new_timestamp

def from_excel_val_to_mins(excel_mins):
    
    offset_mins=str(excel_mins)
    offset_mins=rreplace(string= offset_mins, find=".", replace="", n_occurences=offset_mins.count(".")-1)
    offset_mins = float(offset_mins)
    
    return offset_mins



def get_delayed_psychopy_timestamp(timestamp, offset_mins):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time = dt.datetime.fromtimestamp(timestamp, tz)
    time_change = dt.timedelta(minutes=offset_mins)
    new_time = local_time - time_change
    new_timestamp =  dt.datetime.timestamp(new_time)
    return new_timestamp

## Read, manipulate and slice Shimmer dataframe

In [6]:
#######---------------------------------------------------
###### ------------------Shimmer--------------------------
###----------------------------------------------------------


def read_shimmer_sensor(sensor_file_path):
    
    shimmer_df = pd.read_csv(sensor_file_path, sep='\t', low_memory=False)
    shimmer_df = shimmer_df.reset_index()
    shimmer_df.columns = shimmer_df.iloc[0]
    shimmer_df.drop([0, 1], axis=0, inplace=True)
    shimmer_df=shimmer_df.reset_index(drop=True)
    
    return shimmer_df

def standardize_timestamps_shimmer(shimmer_df, timestamps_col_name):
    
    timesstamps_list = shimmer_df[timestamps_col_name].to_list()
    new_timestamps_list = [float(val)/1000 for val in  timesstamps_list]
    
    shimmer_df[timestamps_col_name] = new_timestamps_list
    
    return shimmer_df



def get_offset_timestamp(timestamp, offset_mins):
    
    time_zone = 'Europe/Berlin'
    tz = pytz.timezone(time_zone)
    local_time = dt.datetime.fromtimestamp(timestamp, tz)
    time_change = dt.timedelta(minutes=offset_mins)
    new_time = local_time + time_change
    new_timestamp =  dt.datetime.timestamp(new_time)
    return new_timestamp

def get_list_timestamp_interest(starting_timestamp, list_offset_mins):
    
    starting_timestamp_list=[]
    
    for offset_min in list_offset_mins:
        starting_timestamp_list.append(starting_timestamp)
        timestamp_offset = get_offset_timestamp(starting_timestamp, offset_min)
        starting_timestamp = timestamp_offset  
     
    
    #starting_timestamp_list = sorted(starting_timestamp_list, key = lambda x:float(x))
    return starting_timestamp_list
        
        

def slice_df_wrt_timestamps(df, start_timestamp, end_timestamp, timestamps_col):
    
    sliced_df=df[(df[timestamps_col]>= start_timestamp) & (df[timestamps_col] <= end_timestamp)]
    
    return sliced_df


def from_str_to_float(str_list):
    #float(recovery_onset_str_timestamps[0][1:-1])
    
    float_array =[float(val[1:-1]) for val in str_list]
    
    return float_array

def col_from_str_float (df, col_name):
    
    str_list = df[col_name].values
    
    float_array =[float(val) for val in str_list]
    
    df[col_name] = float_array
    
    return df

## Time Domain HRV

In [7]:
def detect_peaks(ecg_signal, threshold=0.3, qrs_filter=None):
    '''
    Peak detection algorithm using cross corrrelation and threshold 
    '''
    if qrs_filter is None:
        # create default qrs filter, which is just a part of the sine function
        t = np.linspace(1.5 * np.pi, 3.5 * np.pi, 15)
        qrs_filter = np.sin(t)
    
    # normalize data
    ecg_signal = (ecg_signal - ecg_signal.mean()) / ecg_signal.std()

    # calculate cross correlation
    similarity = np.correlate(ecg_signal, qrs_filter, mode="same")
    similarity = similarity / np.max(similarity)

    # return peaks (values in ms) using threshold
    return ecg_signal[similarity > threshold].index, similarity



def group_peaks(p, threshold=5):
    '''
    The peak detection algorithm finds multiple peaks for each QRS complex. 
    Here we group collections of peaks that are very near (within threshold) and we take the median index 
    '''
    # initialize output
    output = np.empty(0)

    # label groups of sample that belong to the same peak
    peak_groups, num_groups = label(np.diff(p) < threshold)

    # iterate through groups and take the mean as peak index
    for i in np.unique(peak_groups)[1:]:
        peak_group = p[np.where(peak_groups == i)]
        output = np.append(output, int(np.median(peak_group)))
        
        
    #output = int(output)
    return output

def from_rr_ind_2_msec(r_peaks_ind, sampling_freq = 256):
    
    # RR-intervals are the differences between successive peaks
    r_peaks_sec = r_peaks_ind*1/sampling_freq
    r_peaks_msec = r_peaks_sec*1000

    r_peaks_diff_msec=np.diff(r_peaks_msec)
    
    return r_peaks_diff_msec
    
    
    
    
    

def remove_outliers_rri(r_peaks_diff_msec, sampling_freq = 256, outlier_std = 1.5):
    
    # RR-intervals are the differences between successive peaks
    #r_peaks_sec = r_peaks*1/sampling_freq
    #r_peaks_msec = r_peaks_sec*1000

    #r_peaks_diff=np.diff(r_peaks_msec)
    
    rr_corrected = r_peaks_diff_msec.copy()

    rr_corrected[np.abs(zscore(r_peaks_diff_msec)) > outlier_std] = np.median(r_peaks_diff_msec)
    
    return rr_corrected
    


def get_plot_ranges(start=10, end=20, n=5):
    '''
    Make an iterator that divides into n or n+1 ranges. 
    - if end-start is divisible by steps, return n ranges
    - if end-start is not divisible by steps, return n+1 ranges, where the last range is smaller and ends at n
    
    # Example:
    >> list(get_plot_ranges())
    >> [(0.0, 3.0), (3.0, 6.0), (6.0, 9.0)]

    '''
    distance = end - start
    for i in np.arange(start, end, np.floor(distance/n)):
        yield (int(i), int(np.minimum(end, np.floor(distance/n) + i)))
        
        
        
def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    #results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    results['Mean HR (beats/min)'] = np.mean(hr)
    results['STD HR (beats/min)'] = np.std(hr)
    results['Min HR (beats/min)'] = np.min(hr)
    results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
    return results

## Plot raw and filtered ECG

In [8]:
def plot_ecg_wf(sliced_ecg_df,r_peaks, time_offset, dest_fol_path, fig_name,  raw_data_col_name="ecg", 
                filtered_data_col_name="filtered", nr_plots=3, 
                fig_title_raw = "Raw ECG", fig_title_filtered = "Filtered ECG"):    
    
    
    starting_index_slcide_df= sliced_ecg_df.index[0]
    sampfrom = starting_index_slcide_df
    sampto = starting_index_slcide_df+256*time_offset*60
    
    rr_interval_cumm = []
    for start, stop in get_plot_ranges(sampfrom, sampto, nr_plots):
        print(start)
        print(stop)
        
        # get sliced data of ECG
        #cond_slice = (sliced_ecg_df.index>= start) & (sliced_ecg_df.index <= stop)
        #sliced_ecg_hrv =sliced_ecg_df.filtered[cond_slice]
        
        sliced_ecg_hrv = sliced_ecg_df[filtered_data_col_name][(sliced_ecg_df.index >= start) & (sliced_ecg_df.index<= stop)]
        
        sliced_ecg_raw = sliced_ecg_df[raw_data_col_name][(sliced_ecg_df.index >= start) & (sliced_ecg_df.index<= stop)]
        
        
        r_peaks_selected = [r_p for r_p in r_peaks if r_p >= start and r_p <= stop]
        
        fig_ecg=plt.figure(figsize=(20, 15))

        plt.subplot(211)
        plt.title(fig_title_raw)
        plt.plot(sliced_ecg_raw, label="ECG", color="#51A6D8", linewidth=1)
        #plt.plot(sliced_ecg_hrv.index[start: stop], sliced_ecg_hrv, label="ECG", color="#51A6D8", linewidth=1)
        #plt.plot(r_peaks_selected, np.repeat(0.3, len(r_peaks_selected)), label="peaks", color="orange", marker="o", linestyle="None")
        plt.legend(loc="upper right")
        plt.xlabel("Time (milliseconds)")
        plt.ylabel("Amplitude (arbitrary unit)")

        plt.subplot(212)
        plt.title(fig_title_filtered)
        
        #plt.plot(ecg_slice.index, similarity, label="Similarity with QRS filter", color="olive", linewidth=1)
        plt.plot(sliced_ecg_hrv, label="Filtered ECG", color="olive", linewidth=1)
        plt.plot(r_peaks_selected, np.repeat(0.2, len(r_peaks_selected)), label="peaks", color="orange", marker="o", linestyle="None")
        plt.legend(loc="upper right")
        plt.xlabel("Time (milliseconds)")
        plt.ylabel("Similarity (normalized)")
        
        plt.show()
        plot_figure = []
        plot_figure = fig_name
        print(plot_figure)
        #print(plot_figure[0])
        fig_name = str(start) + "_" + str(stop) + "_" + plot_figure
        fig_path = os.path.join(dest_fol_path, fig_name)
        fig_ecg.savefig(fig_path)
        
        

### Plot mean and standard deviations

In [9]:
def plot_mean_std(mean_list, std_list, label_list, title_figure, y_label_fig):
    x_pos = np.arange(len(label_list))
    fig, ax = plt.subplots()
    ax.bar(x_pos, mean_list, yerr=std_list, align='center', alpha=0.5, capsize=10 )
    ax.set_ylabel(y_label_fig)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(label_list, rotation=45, ha='right', rotation_mode='anchor')
    #title_ax = "VP001_251023_Session1_Shimmer_FBA7_Calibrated_SD"
    ax.set_title(title_figure)
    ax.yaxis.grid(True)
    #plt.ylim(0, 1) 
    plt.tight_layout()
    plt.show()
    return fig

## Min-Max Normaliaztion

In [10]:
def min_max_norm(all_rr_one_list, all_rr_sublist):
    
    mean_norm_list =[]
    std_norm_list = []
    
    all_rr_one_list = np.array(all_rr_one_list)
    max_val = np.max(all_rr_one_list)
    min_val =np.min(all_rr_one_list)
    
    for sel_list in all_rr_sublist:
        
        sel_list_np = np.array(sel_list)
        
        range_list = max_val-min_val
        
        sel_lis_np_sub = sel_list_np - min_val
        
        sel_lis_norm= sel_lis_np_sub/range_list
        
        mean_norm = np.mean(sel_lis_norm)
        mean_norm_list.append(mean_norm)
        
        std_norm  = np.std(sel_lis_norm)   
        std_norm_list.append(std_norm)
        #print(mean_norm)
        #print(std_norm)
                
        
    return mean_norm_list, std_norm_list

In [11]:
def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    #results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    results['Mean HR (beats/min)'] = np.mean(hr)
    results['STD HR (beats/min)'] = np.std(hr)
    results['Min HR (beats/min)'] = np.min(hr)
    results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
    return results

## ----- Cumm Data

In [15]:
teaa_data_main_dir ="/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/dataset_sony_TEAA/"
data_dir_path = os.path.join(teaa_data_main_dir, "data")
event_dir = os.path.join(teaa_data_main_dir, "Psychopy_events")
sub_dirs=next(os.walk(data_dir_path))[1]
sub_dirs=sorted(sub_dirs)

In [16]:
sub_dirs

['WP2_P01_FQPD',
 'WP2_P02_YPCK',
 'WP2_P03_RFIB',
 'WP2_P04_TUYR',
 'WP2_P05_MCUW',
 'WP2_P06_UUAH',
 'WP2_P07_VCIP',
 'WP2_P08_NYPX',
 'WP2_P09_CWAP',
 'WP2_P10_DNEE',
 'WP2_P11_TXTU',
 'WP2_P12_FMDS',
 'WP2_P13_NYBN',
 'WP2_P14_WHVF',
 'WP2_P15_MDTP',
 'WP2_P16_MFFX',
 'WP2_P17_EKQV',
 'WP2_P18_HSSW',
 'WP2_P19_UHTF',
 'WP2_P20_OBVP',
 'WP2_P21_OWBX',
 'WP2_P22_RIXP',
 'WP2_P23_JQPD',
 'WP2_P24_PPTC',
 'WP2_P25_CMRG',
 'WP2_P26_IRJH',
 'WP2_P27_VJBM',
 'WP2_P28_OSKE',
 'WP2_P29_GMIP',
 'WP2_P30_ZCTR',
 'WP2_P31_WDDN']

In [17]:
sampling_freq_shimmer = 256
event_offset_min = 5
time_zone = 'Europe/Berlin'
tz = pytz.timezone(time_zone)

for sub_dir in sub_dirs:
    ###----------------------------------------------
    #### reading psychopy events, shimmer files
    ##----------------------------------------------
    
    
    #print(psychopy_file_path)
    sub_dir_path= os.path.join(data_dir_path, sub_dir)
    
    #events_path_list = os.path.join(event_dir, "Psychopy_events")
    events_path_list = glob.glob(event_dir+ "/*.csv")
    
    for sel_event_path in events_path_list:
        fol_path, file_name_ii=os.path.split(sel_event_path)
        #print(file_name_ii)
        if sub_dir in file_name_ii:
            sel_event_file = sel_event_path
            
            try:
                psy_events_df = pd.read_csv(sel_event_file, sep=";", index_col="Unnamed: 0")
            except:
                psy_events_df = pd.read_csv(sel_event_file, index_col="Unnamed: 0")
            
            print(sel_event_file)
            
            fol_path_tri, file_name_events=os.path.split(sel_event_file)
            
            #print(file_name_events)
            
    shimmer_files_subdir = os.path.join(sub_dir_path, "Shimmer")
    shimmer_files_subdir_subdir_name = next(os.walk(shimmer_files_subdir))[1][0]
    shimmer_files_parent_path = os.path.join(shimmer_files_subdir, shimmer_files_subdir_subdir_name)
    shimmer_files_name = glob.glob(shimmer_files_parent_path+ "/*.csv")
    
    for sh_file in shimmer_files_name:
        
        if '_DA83_' in sh_file:
            ecg_shimmer_path = sh_file
            print("shimmer file path: " + ecg_shimmer_path)
            print("-----------------------------")
            shimmer_timestamp_col_name = "Shimmer_DA83_Timestamp_Unix_CAL"
            ecg_col_name = "Shimmer_DA83_ECG_LA-RA_24BIT_CAL"
            
            
        elif "_8943_" in sh_file:
            ecg_shimmer_path=sh_file
            print("shimmer file path: " + ecg_shimmer_path)
            print("-----------------------------")
            shimmer_timestamp_col_name = "Shimmer_8943_Timestamp_Unix_CAL"
            ecg_col_name = "Shimmer_8943_ECG_LA-RA_24BIT_CAL"
            
            
    shimmer_df=read_shimmer_sensor(ecg_shimmer_path)    
    selected_ecg_df = shimmer_df[[shimmer_timestamp_col_name, ecg_col_name]]
    selected_ecg_df = selected_ecg_df.copy()
    selected_ecg_df=standardize_timestamps_shimmer(selected_ecg_df, shimmer_timestamp_col_name)
    selected_ecg_df['ecg'] = selected_ecg_df[ecg_col_name]
    selected_ecg_df=col_from_str_float(selected_ecg_df, "ecg")
    
    sh_ts_list = selected_ecg_df[shimmer_timestamp_col_name].values
    shimmer_start_time = dt.datetime.fromtimestamp(sh_ts_list[0], tz)
    print("shimmer first ts: "+ str(shimmer_start_time))
    shimmer_stop_time = dt.datetime.fromtimestamp(sh_ts_list[-1], tz)
    print("shimmer last ts: "+ str(shimmer_stop_time))
    print("-------------------------------------------")
            
            
    
        
        
    pa_row=psy_events_df.loc["pa"]
    b1_row=psy_events_df.loc["b1"]
    b2_row=psy_events_df.loc["b2"]
    block_1_type=psy_events_df.loc["b1_type"]["block_type"]
    block_2_type=psy_events_df.loc["b2_type"]["block_type"]
    #delay_mins = pa_row["offset"]
    
    baseline_onset_pa=pa_row["baseline_onset"]
    baseline_onset_pa=from_excel_val_to_timestamp(baseline_onset_pa)
    
    baseline_onset_b1=b1_row["baseline_onset"]
    baseline_onset_b1=from_excel_val_to_timestamp(baseline_onset_b1)
    baseline_onset_b2=b2_row["baseline_onset"]
    baseline_onset_b2=from_excel_val_to_timestamp(baseline_onset_b2)

    baseline_onset_timestamps = []
    baseline_onset_timestamps.append(baseline_onset_pa)
    baseline_onset_timestamps.append(baseline_onset_b1)
    baseline_onset_timestamps.append(baseline_onset_b2)

    task_onset_pa = pa_row["task_onset"]
    task_onset_pa=from_excel_val_to_timestamp(task_onset_pa)
    task_onset_b1=b1_row["task_onset"]
    task_onset_b1=from_excel_val_to_timestamp(task_onset_b1)
    task_onset_b2 = b2_row["task_onset"]
    task_onset_b2=from_excel_val_to_timestamp(task_onset_b2)

    task_onset_float_timestamp = []
    task_onset_float_timestamp.append(task_onset_pa)
    task_onset_float_timestamp.append(task_onset_b1)
    task_onset_float_timestamp.append(task_onset_b2)

    recovery_onset_pa = pa_row["recovery_onset"]
    recovery_onset_pa=from_excel_val_to_timestamp(recovery_onset_pa)
    recovery_onset_b1=b1_row["recovery_onset"]
    recovery_onset_b1=from_excel_val_to_timestamp(recovery_onset_b1)
    recovery_onset_b2 = b2_row["recovery_onset"]
    recovery_onset_b2=from_excel_val_to_timestamp(recovery_onset_b2)

    recovery_onset_list_timestamps = []
    recovery_onset_list_timestamps.append(recovery_onset_pa)
    recovery_onset_list_timestamps.append(recovery_onset_b1)
    recovery_onset_list_timestamps.append(recovery_onset_b2)
    
    
    delay_mins= from_excel_val_to_mins(pa_row["offset"])
    print("Delay for this recording sesssion is: " + str(delay_mins)+ " mins")
    print("----------------------------------")
    
    ####----------For Debugging: printing the times for all the events in one recording
    
    for ind_ii, sel_baseline in enumerate(baseline_onset_timestamps):

        baseline_ts = sel_baseline
        baseline_time_delayed =dt.datetime.fromtimestamp(baseline_ts, tz)
        print("baseline time before correction: "+str(baseline_time_delayed))
        baseline_ts = get_delayed_psychopy_timestamp(baseline_ts, delay_mins)
        baseline_time = dt.datetime.fromtimestamp(baseline_ts, tz)
        print("baseline time (corrected): "+str(baseline_time))
        ##------------------------------------------
        
        task_ts = task_onset_float_timestamp[ind_ii]
        task_time_delayed =dt.datetime.fromtimestamp(task_ts, tz)
        print("task time before correction: "+str(task_time_delayed))
        
        task_ts = get_delayed_psychopy_timestamp(task_ts, delay_mins)
        task_time = dt.datetime.fromtimestamp(task_ts, tz)
        print("task time (corrected): "+str(task_time))
        ##---------------------------------------------------
        recovery_ts = recovery_onset_list_timestamps[ind_ii]
        
        recovery_time_delayed =dt.datetime.fromtimestamp(recovery_ts, tz)
        print("recovery time before correction: "+str(recovery_time_delayed))
        
        recovery_ts = get_delayed_psychopy_timestamp(recovery_ts, delay_mins)
        recovery_time = dt.datetime.fromtimestamp(recovery_ts, tz)

        print("recovery time (corrected): "+str(recovery_time))

        print("---------------------------------------------")
        
        
        
    for ind_timestamp, sel_baseline in enumerate(baseline_onset_timestamps):
        task_type_list = []
        all_ibi_one_list_one_task = []
        all_ibi_sublist_one_task = []
        mean_cumm_list = []
        std_cumm_list = []
        
        mean_hbpm_cumm_list = []
        std_hbpm_cumm_list = []
        
        if ind_timestamp==0:
            task_type_ii = "pre_assesment"
        
        elif ind_timestamp==1:
            if block_1_type=="negative":
                task_type_ii = "negative"
            elif block_1_type =="positive":
                task_type_ii = "positive"
        
        elif ind_timestamp==2:
            if block_1_type=="positive":
                task_type_ii = "negative"
            elif block_1_type =="negative":
                task_type_ii = "positive"
                
                
        print(task_type_ii)
        task_type_list.append(task_type_ii)           
        start_timestamp_baseline = sel_baseline
        stop_timestamp_baseline = get_offset_timestamp(start_timestamp_baseline, event_offset_min)
        
        #sliced_ecg_df_baseline=slice_df_wrt_timestamps(selected_ecg_df, start_timestamp_baseline, stop_timestamp_baseline, shimmer_timestamp_col_name)
        #sliced_ecg_df_baseline=sliced_ecg_df_baseline.reset_index(drop=True)
        #ecg_array_baseline = sliced_ecg_df_baseline["ecg"].values
        #out_baseline = ecg.ecg(signal=ecg_array_baseline, sampling_rate=sampling_freq_shimmer, show=False, interactive=False)
        #sliced_ecg_df_baseline["filtered"] = out_baseline["filtered"]
        #r_peaks_ind_baseline = out_baseline["rpeaks"]
        #rr_msec_baseline=from_rr_ind_2_msec(r_peaks_ind_baseline)
        #rr_msec_baseline_clean=remove_outliers_rri(rr_msec_baseline, sampling_freq = 256, outlier_std = 1.5)
        
        #time_domain_features=timedomain(rr_msec_baseline_clean)
        
        ###--------------------------------------------------
        #mean_hbpm_baseline = time_domain_features["Mean HR (beats/min)"]
        #std_hbpm_baseline = time_domain_features["STD HR (beats/min)"]
        
        #all_ibi_one_list_one_task.extend(rr_msec_baseline_clean)
        #all_ibi_sublist_one_task.append(rr_msec_baseline_clean)
        #mean_ibi = np.mean(rr_msec_baseline_clean)
        #mean_cumm_list.append(np.mean(rr_msec_baseline_clean))
        #std_cumm_list.append(np.std(rr_msec_baseline_clean))
        
        #mean_hbpm_cumm_list.append(mean_hbpm_baseline)
        #std_hbpm_cumm_list.append(std_hbpm_baseline)
        
         #print("BASELINE DATA")
        #fig_name = "Baseline_"+ sub_id + ".png"
        #fig_name = "Baseline"+str(ind_timestamp)+".png"
        #plt_title_raw = "Raw ECG: Baseline"
        #plt_title_filtered = "Filtered ECG: Baseline"
        #plot_ecg_wf(sliced_ecg_df_baseline,r_peaks_ind_baseline,offset_min,dest_path_save_figures, fig_name, nr_plots=total_plots, fig_title_raw=plt_title_raw, fig_title_filtered=plt_title_filtered)
        
        
        ####--------------------------------------------------------------------
        ####------------------for debugging purposes----------------------------
        ####--------------------------------------------------------------------
        #start_time = dt.datetime.fromtimestamp(start_timestamp_baseline, tz)
        #end_time  = dt.datetime.fromtimestamp(stop_timestamp_baseline, tz)

        #print("baseline start time: "+str(start_time))
        #print("baseline stop time: "+str(end_time))

        #time_stamps_shimmer= sliced_ecg_df_baseline[shimmer_timestamp_col_name].values

        #start_timestamp_shimmer = time_stamps_shimmer[0]
        #end_timestamp_shimmer =  time_stamps_shimmer[-1]

        #start_time = dt.datetime.fromtimestamp(start_timestamp_shimmer, tz)
        #end_time  = dt.datetime.fromtimestamp(end_timestamp_shimmer, tz)

        #print("shimmer_baseline start time: "+str(start_time))
        #print("shimmer_baseline stop time: "+str(end_time))
        ###-----------------------------------------------------------------
        
        ###-----------------------------------------------------------
        ###---------  Task -------------------------------------------
        ###-----------------------------------------------------------
        
        
        
        start_timestamp_task= task_onset_float_timestamp[ind_timestamp]
        stop_timestamp_task = get_offset_timestamp(start_timestamp_task, event_offset_min)
        #sliced_ecg_df_task=slice_df_wrt_timestamps(selected_ecg_df, start_timestamp_task, stop_timestamp_task, shimmer_timestamp_col_name)
        #sliced_ecg_df_task=sliced_ecg_df_task.reset_index(drop=True)
        
        
        ###-------------------------------------------------
        #ecg_array_task = sliced_ecg_df_task["ecg"].values
        #out_task = ecg.ecg(signal=ecg_array_task, sampling_rate=sampling_freq_shimmer, show=False, interactive=False)
        #sliced_ecg_df_task["filtered"] = out_task["filtered"]
        #r_peaks_ind_task = out_task["rpeaks"]
        #rr_msec_task=from_rr_ind_2_msec(r_peaks_ind_task)
        #rr_msec_task_clean=remove_outliers_rri(rr_msec_task, sampling_freq = 256, outlier_std = 1.5)
        
        #time_domain_features=timedomain(rr_msec_task_clean)
        ###--------------------------------------------------
        #mean_hbpm_task = time_domain_features["Mean HR (beats/min)"]
        #std_hbpm_task = time_domain_features["STD HR (beats/min)"]
        
        #all_ibi_one_list_one_task.extend(rr_msec_task_clean)
        #all_ibi_sublist_one_task.append(rr_msec_task_clean)
        
        #mean_cumm_list.append(np.mean(rr_msec_task_clean))
        #std_cumm_list.append(np.std(rr_msec_task_clean))
        
        #mean_hbpm_cumm_list.append(mean_hbpm_task)
        #std_hbpm_cumm_list.append(std_hbpm_task)
        
        #print("TASK DATA")
        #fig_name = "Task"+ sub_id + ".png"
        #fig_name = "Task"+str(ind_timestamp)+".png"
        #plt_title_raw = "Raw ECG: Task"
        #plt_title_filtered = "Filtered ECG: Task"
        #plot_ecg_wf(sliced_ecg_df_task,r_peaks_ind_task,offset_min, dest_path_save_figures,fig_name, 
         #           nr_plots=total_plots, fig_title_raw=plt_title_raw, fig_title_filtered=plt_title_filtered)
            
        
         #####------------------------------------------------------------
        ##### -------------for debugging---------------------------------
        #####------------------------------------------------------------

        start_time = dt.datetime.fromtimestamp(start_timestamp_task, tz)
        end_time  = dt.datetime.fromtimestamp(stop_timestamp_task, tz)

        print("task start time: "+str(start_time))
        print("task stop time: "+str(end_time))

        #time_stamps_shimmer= sliced_ecg_df_task[shimmer_timestamp_col_name].values

        #start_timestamp_shimmer = time_stamps_shimmer[0]
        #end_timestamp_shimmer =  time_stamps_shimmer[-1]

        #start_time = dt.datetime.fromtimestamp(start_timestamp_shimmer, tz)
        #end_time  = dt.datetime.fromtimestamp(end_timestamp_shimmer, tz)

        #print("shimmer_task start time: "+str(start_time))
        #print("shimmer_task stop time: "+str(end_time))
        
        
        
        
        
        ####-------------------------------------------------------------
        ####-----------------Recovery--------------------------------------
        ####-------------------------------------------------------------
    
        start_timestamp_recovery= recovery_onset_list_timestamps[ind_timestamp]
        stop_timestamp_recovery =get_offset_timestamp(start_timestamp_recovery, event_offset_min)
        #sliced_ecg_df_recovery=slice_df_wrt_timestamps(selected_ecg_df, start_timestamp_recovery, stop_timestamp_recovery, shimmer_timestamp_col_name)
        #sliced_ecg_df_recovery=sliced_ecg_df_recovery.reset_index(drop=True)

        ###-------------------------------------------------
        #ecg_array_recovery = sliced_ecg_df_recovery["ecg"].values
        #out_recovery = ecg.ecg(signal=ecg_array_recovery, sampling_rate=sampling_freq_shimmer, show=False, interactive=False)
        #sliced_ecg_df_recovery["filtered"] = out_recovery["filtered"]
        #r_peaks_ind_recovery = out_recovery["rpeaks"]
        #rr_msec_recovery=from_rr_ind_2_msec(r_peaks_ind_recovery)
        #rr_msec_recovery_clean=remove_outliers_rri(rr_msec_recovery, sampling_freq = 256, outlier_std = 1.5)
        #time_domain_features=timedomain(rr_msec_recovery_clean)
        ###--------------------------------------------------
        #mean_hbpm_recovery = time_domain_features["Mean HR (beats/min)"]
        #std_hbpm_recovery = time_domain_features["STD HR (beats/min)"]
        ###-----------------------------------------------------
        #all_ibi_one_list_one_task.extend(rr_msec_task_clean)
        #all_ibi_sublist_one_task.append(rr_msec_task_clean)
        #mean_cumm_list.append(np.mean(rr_msec_recovery_clean))
        #std_cumm_list.append(np.std(rr_msec_recovery_clean))
        
        #mean_hbpm_cumm_list.append(mean_hbpm_recovery)
        #std_hbpm_cumm_list.append(std_hbpm_recovery)
        
        
        #####------------------------------------------------------------
        ##### -------------for debugging---------------------------------
        #####------------------------------------------------------------
        start_time = dt.datetime.fromtimestamp(start_timestamp_recovery, tz)
        end_time  = dt.datetime.fromtimestamp(stop_timestamp_recovery, tz)

        print("recovery start time: "+str(start_time))
        print("recovery stop time: "+str(end_time))


        #time_stamps_shimmer= sliced_ecg_df_recovery[shimmer_timestamp_col_name].values

        #start_timestamp_shimmer = time_stamps_shimmer[0]
        #end_timestamp_shimmer =  time_stamps_shimmer[-1]

        #start_time = dt.datetime.fromtimestamp(start_timestamp_shimmer, tz)
        #end_time  = dt.datetime.fromtimestamp(end_timestamp_shimmer, tz)

        #print("shimmer_recovery start time: "+str(start_time))
        #print("shimmer_recovery stop time: "+str(end_time))
        
        
        
        
        ####-------------------------------------------------------------
        ####---------------Saving the extracted information--------------
        ####-------------------------------------------------------------
        
        
        #hrv_data_dict = {}
        
        #hrv_data_dict["subject_id"] = file_name_ii
        
        #event_interets_plot = ["Baseline", "Task", "Recovery"]
    
        #hrv_data_dict["event_interst"] = event_interets_plot
    
        #hrv_data_dict["all_rr_one_list"] = all_ibi_one_list_one_task
    
        #hrv_data_dict["all_rr_sublists"] = all_ibi_sublist_one_task
        
        #hrv_data_dict["mean_hbpm"] = mean_hbpm_cumm_list
        #hrv_data_dict["std_hbpmy"] = std_hbpm_cumm_list
        
        #hrv_data_dict["mean_ibi"] = mean_cumm_list
        #hrv_data_dict["std_ibi"] = std_cumm_list
        
        #hrv_data_dict["task_type"] = task_type_ii
        
        
        #pkl_file_hrv = file_name_trig[:-4] + "_"+task_type_ii+ ".pkl"
        
        #print(pkl_file_hrv)
        
        #dest_path_pkl_file = os.path.join(dest_path, pkl_file_hrv)
        #with open(dest_path_pkl_file, "wb") as f:
         #   pickle.dump(hrv_data_dict, f)
          #  f.close()
            
        #title_fig =  task_type_ii+ ":" + file_name_trig[:-4]
        #fig_name =  file_name_trig[:-4]+ "_" + task_type_ii+".png"
        #fig_dest_path = os.path.join(dest_path,fig_name)
        #y_label_fig = "HBPM"
        #fig_mean_std= plot_mean_std(mean_hbpm_cumm_list, std_hbpm_cumm_list, event_interets_plot, title_fig, y_label_fig)
        
        #fig_mean_std.savefig(fig_dest_path)
        
        
        
        #mean_norm_list, std_norm_list = min_max_norm(all_ibi_one_list_one_task, all_ibi_sublist_one_task)
        
        #title_fig =  file_name_ii[:-4] + task_type_ii 
        #y_label_fig = "Norm-IBI"
        #fig_norm_mean_std= plot_mean_std(mean_norm_list, std_norm_list, event_interets_plot, title_fig, y_label_fig)
        
        
        
    
        ###-------------------------------------------------------------------
        ####-------------------for raw and filtered plotting-------------------
        ####-------------------------------------------------------------------
        #print("RECOVERY DATA")
        #fig_name="Recovery_"+ sub_id + ".png"
        #fig_name = "Recovery"+str(ind_timestamp)+".png"
        #plt_title_raw = "Raw ECG: Recovery"
        #plt_title_filtered = "Filtered ECG: Recovery"
        #plot_ecg_wf(sliced_ecg_df_recovery,r_peaks_ind_recovery,offset_min, dest_path_save_figures, 
         #           fig_name, nr_plots=total_plots, fig_title_raw=plt_title_raw, fig_title_filtered=plt_title_filtered )

        ####-------------------------------------------------------------------------
    


        
        
        
        
        
        
        
                
                
        
        
        
        
        
      
    

    
    
                
            
            

/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/dataset_sony_TEAA/Psychopy_events/psychopy_events_WP2_P01_FQPD.csv
shimmer file path: /media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/dataset_sony_TEAA/data/WP2_P01_FQPD/Shimmer/2023-02-08_10.48.36_WP2_P01_FQPD_SD_Session1/WP2_P01_FQPD_Session1_Shimmer_DA83_Calibrated_SD.csv
-----------------------------
shimmer first ts: 2023-02-08 10:53:17.144684+01:00
shimmer last ts: 2023-02-08 12:03:34.922028+01:00
-------------------------------------------
Delay for this recording sesssion is: 42.8808236122131 mins
----------------------------------
baseline time before correction: 2023-02-08 11:03:16.433510+01:00
baseline time (corrected): 2023-02-08 10:20:23.584093+01:00
task time before correction: 2023-02-08 11:08:17.019720+01:00
task time (corrected): 2023-02-08 10:25:24.170303+01:00
recovery time before correction: 2023-02-08 11:13:18.031180+01:00
recovery time (corrected): 2023-02-08 10:30:25.181763+01:00
--------------------------

In [14]:
path_scv ="/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/dataset_sony_TEAA/data/WP2_P07_VCIP/Shimmer/2023-02-10_12.18.50_WP2_P07_VCIP_SD_Session1/WP2_P07_VCIP_Session1_Shimmer_8943_Calibrated_SD.csv"

In [15]:
shimmer_df=read_shimmer_sensor(path_scv) 

In [16]:
shimmer_df

,Shimmer_8943_Timestamp_Unix_CAL,Shimmer_8943_Accel_LN_X_CAL,Shimmer_8943_Accel_LN_Y_CAL,Shimmer_8943_Accel_LN_Z_CAL,Shimmer_8943_ECG_EMG_Status1_CAL,Shimmer_8943_ECG_EMG_Status2_CAL,Shimmer_8943_ECG_IBI_LA_RA_CAL,Shimmer_8943_ECG_IBI_LL_LA_CAL,Shimmer_8943_ECG_IBI_LL_RA_CAL,Shimmer_8943_ECG_IBI_Vx_RL_CAL,Shimmer_8943_ECG_LA-RA_24BIT_CAL,Shimmer_8943_ECG_LL-LA_24BIT_CAL,Shimmer_8943_ECG_LL-RA_24BIT_CAL,Shimmer_8943_ECG_Vx-RL_24BIT_CAL,Shimmer_8943_ECGtoHR_LA_RA_CAL,Shimmer_8943_ECGtoHR_LL_LA_CAL,Shimmer_8943_ECGtoHR_LL_RA_CAL,Shimmer_8943_ECGtoHR_Vx_RL_CAL,NaN
0,1.676031474351715E12,-4.173913043478261,4.315217391304348,8.184782608695652,128.0,128.0,-1.0,-1.0,-1.0,-1.0,14.540225212600854,4.463751848191242,19.003977060792096,-11.121732130257145,-1.0,-1.0,-1.0,-1.0,NaN
1,1.6760314743595276E12,-4.173913043478261,4.33695652173913,8.16304347826087,128.0,128.0,-1.0,-1.0,-1.0,-1.0,14.566621728732793,4.481133160726211,19.047754889459004,-11.1221648600298,-1.0,-1.0,-1.0,-1.0,NaN
2,1.6760314743634338E12,-4.173913043478261,4.3478260869565215,8.184782608695652,128.0,128.0,-1.0,-1.0,-1.0,-1.0,14.559698052370317,4.478608903719058,19.038306956089375,-11.148200768017862,-1.0,-1.0,-1.0,-1.0,NaN
3,1.67603147436734E12,-4.184782608695652,4.3478260869565215,8.184782608695652,128.0,128.0,-1.0,-1.0,-1.0,-1.0,14.518155994195462,4.469593700122081,18.987749694317543,-11.139401929307216,-1.0,-1.0,-1.0,-1.0,NaN
4,1.6760314743712463E12,-4.173913043478261,4.326086956521739,8.184782608695652,128.0,128.0,-1.0,-1.0,-1.0,-1.0,14.515126885786879,4.4693773352357535,18.984504221022632,-11.130963698740446,-1.0,-1.0,-1.0,-1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049841,1.6760355802228088E12,-4.217391304347826,3.5978260869565215,8.326086956521738,128.0,128.0,-1.0,-1.0,-1.0,-1.0,6.5516730012503865,1.614659025032405,8.166332026282792,-10.459871942981714,65.08474576271186,65.36170212765957,65.08474576271186,65.08474576271186,NaN
1049842,1.676035580226715E12,-4.25,3.608695652173913,8.315217391304348,128.0,128.0,-1.0,-1.0,-1.0,-1.0,6.5156843084912675,1.6263427288940813,8.142027037385349,-10.456698591315579,65.08474576271186,65.36170212765957,65.08474576271186,65.08474576271186,NaN
1049843,1.6760355802306213E12,-4.25,3.619565217391304,8.293478260869565,128.0,128.0,-1.0,-1.0,-1.0,-1.0,6.503279388341832,1.6323288240824718,8.135608212424303,-10.473430809191562,65.08474576271186,65.36170212765957,65.08474576271186,65.08474576271186,NaN
1049844,1.6760355802345276E12,-4.25,3.619565217391304,8.282608695652174,128.0,128.0,-1.0,-1.0,-1.0,-1.0,6.508760632128792,1.6329057971126781,8.14166642924147,-10.45994406461049,65.08474576271186,65.36170212765957,65.08474576271186,65.08474576271186,NaN


# ----------------------------------------------------------
## Cummulative --------------------------------------------
# ---------------------------------------------------------

In [ ]:
data_dir_path = "/media/muhammad/d5e4618d-a960-4f0d-a521-1c804bbebbb4/dataset_sony_sam_2024/WP2"
sub_dirs=next(os.walk(data_dir_path))[1]
sub_dirs=sorted(sub_dirs)

In [ ]:
dest_path = "/home/muhammad/Desktop/Datasets/dataset_sony_sam/Data_analysis_ECG/pkld_file_per_subject_per_task"

In [ ]:
sampling_freq_shimmer = 256
offset_min = 5
time_zone = 'Europe/Berlin'
tz = pytz.timezone(time_zone)

for sub_dir in sub_dirs:
    ###-----------------------------------------------
    #### reading psychopy, events, shimmer files
    ##----------------------------------------------
    
    sub_dir_path= os.path.join(data_dir_path, sub_dir)
    psychopy_file_path = os.path.join(sub_dir_path, "Psychopy")
    psychopy_file_path = glob.glob(psychopy_file_path+ "/*.csv")[0]
    #print(psychopy_file_path)
    events_path_list = os.path.join(sub_dir_path, "preAnalysis")
    events_path_list = glob.glob(events_path_list+ "/*.csv")
    
    for sel_event_path in events_path_list:
        fol_path, file_name_ii=os.path.split(sel_event_path)
        #print(file_name_ii)
        if "trigger" in file_name_ii:
            sel_trigger_file = sel_event_path
            
            fol_path_tri, file_name_trig=os.path.split(sel_trigger_file)
            
            print(file_name_trig)
            
    shimmer_files_subdir = os.path.join(sub_dir_path, "Shimmer")
    shimmer_files_subdir_subdir_name = next(os.walk(shimmer_files_subdir))[1][0]
    shimmer_files_parent_path = os.path.join(shimmer_files_subdir, shimmer_files_subdir_subdir_name)
    shimmer_files_name = glob.glob(shimmer_files_parent_path+ "/*.csv")
    
    for sh_file in shimmer_files_name:
        
        if '_8943_' in sh_file:
            ecg_shimmer_path = sh_file
            shimmer_timestamp_col_name = "Shimmer_8943_Timestamp_Unix_CAL"
            ecg_col_name = "Shimmer_8943_ECG_LA-RA_24BIT_CAL"
        elif "_8108_" in sh_file:
            ecg_shimmer_path=sh_file
            shimmer_timestamp_col_name = "Shimmer_8108_Timestamp_Unix_CAL"
            ecg_col_name = "Shimmer_8108_ECG_LA-RA_24BIT_CAL"
            
            
            
            
            
            
    psychopy_df = pd.read_csv(psychopy_file_path)
    block_1_type = psychopy_df["block1"][0]
    #print(block_1_type)
    
    event_timestamps_df = pd.read_csv(sel_trigger_file)
    
    baseline_onset = event_timestamps_df["baseline_onset"].values
    baseline_onset_timestamps = baseline_onset[1:]
    task_onset =event_timestamps_df["task_onset"].values
    task_onset_str_timestamps = task_onset[1:]
    task_onset_float_timestamp = from_str_to_float(task_onset_str_timestamps)
    recovery_onset =event_timestamps_df["recovery_onset"].values
    recovery_onset_str_timestamps = recovery_onset[1:]
    recovery_onset_list_timestamps=from_str_to_float(recovery_onset_str_timestamps)
    
    
    shimmer_df=read_shimmer_sensor(ecg_shimmer_path)
    
    selected_ecg_df = shimmer_df[[shimmer_timestamp_col_name, ecg_col_name]]
    selected_ecg_df = selected_ecg_df.copy()
    selected_ecg_df=standardize_timestamps_shimmer(selected_ecg_df, shimmer_timestamp_col_name)
    selected_ecg_df['ecg'] = selected_ecg_df[ecg_col_name]
    selected_ecg_df=col_from_str_float(selected_ecg_df, "ecg")
    
    ###-------------------------------------------------------
    ####-----------------------------------------------------
    ###-------------------------------------------------------
    
    #all_ibi_one_list_one_task = []
    #all_ibi_sublist_one_task = []
    
    
    for ind_timestamp, sel_baseline in enumerate(baseline_onset_timestamps):
        task_type_list = []
        all_ibi_one_list_one_task = []
        all_ibi_sublist_one_task = []
        mean_cumm_list = []
        std_cumm_list = []
        
        mean_hbpm_cumm_list = []
        std_hbpm_cumm_list = []
        
        if ind_timestamp==0:
            task_type_ii = "pre_assesment"
        
        elif ind_timestamp==1:
            if block_1_type=="negative":
                task_type_ii = "negative"
            elif block_1_type =="positive":
                task_type_ii = "positive"
        
        elif ind_timestamp==2:
            if block_1_type=="positive":
                task_type_ii = "negative"
            elif block_1_type =="negative":
                task_type_ii = "positive"
                
        #elif ind_timestamp==2:
        #    if block_1_type=="negative":
        #       task_type_ii = "positive"
        #    elif block_1_type =="positive":
        #        task_type_ii = "negative"
                
        print(task_type_ii)
        task_type_list.append(task_type_ii)           
        start_timestamp_baseline = sel_baseline
        stop_timestamp_baseline = get_offset_timestamp(start_timestamp_baseline, offset_min)


        sliced_ecg_df_baseline=slice_df_wrt_timestamps(selected_ecg_df, start_timestamp_baseline, stop_timestamp_baseline, shimmer_timestamp_col_name)
        sliced_ecg_df_baseline=sliced_ecg_df_baseline.reset_index(drop=True)
        ecg_array_baseline = sliced_ecg_df_baseline["ecg"].values
        out_baseline = ecg.ecg(signal=ecg_array_baseline, sampling_rate=sampling_freq_shimmer, show=False, interactive=False)
        sliced_ecg_df_baseline["filtered"] = out_baseline["filtered"]
        r_peaks_ind_baseline = out_baseline["rpeaks"]
        rr_msec_baseline=from_rr_ind_2_msec(r_peaks_ind_baseline)
        rr_msec_baseline_clean=remove_outliers_rri(rr_msec_baseline, sampling_freq = 256, outlier_std = 1.5)
        
        time_domain_features=timedomain(rr_msec_baseline_clean)
        ###--------------------------------------------------
        mean_hbpm_baseline = time_domain_features["Mean HR (beats/min)"]
        std_hbpm_baseline = time_domain_features["STD HR (beats/min)"]
        
        
        
        all_ibi_one_list_one_task.extend(rr_msec_baseline_clean)
        all_ibi_sublist_one_task.append(rr_msec_baseline_clean)
        #mean_ibi = np.mean(rr_msec_baseline_clean)
        mean_cumm_list.append(np.mean(rr_msec_baseline_clean))
        std_cumm_list.append(np.std(rr_msec_baseline_clean))
        
        mean_hbpm_cumm_list.append(mean_hbpm_baseline)
        std_hbpm_cumm_list.append(std_hbpm_baseline)
    
        #print("BASELINE DATA")
        #fig_name = "Baseline_"+ sub_id + ".png"
        #fig_name = "Baseline"+str(ind_timestamp)+".png"
        #plt_title_raw = "Raw ECG: Baseline"
        #plt_title_filtered = "Filtered ECG: Baseline"
        #plot_ecg_wf(sliced_ecg_df_baseline,r_peaks_ind_baseline,offset_min,dest_path_save_figures, fig_name, nr_plots=total_plots, fig_title_raw=plt_title_raw, fig_title_filtered=plt_title_filtered)
    
    
    
        ####--------------------------------------------------------------------
        ####------------------for debugging purposes----------------------------
        ####--------------------------------------------------------------------
        start_time = dt.datetime.fromtimestamp(start_timestamp_baseline, tz)
        end_time  = dt.datetime.fromtimestamp(stop_timestamp_baseline, tz)

        print("baseline start time: "+str(start_time))
        print("baseline stop time: "+str(end_time))

        time_stamps_shimmer= sliced_ecg_df_baseline[shimmer_timestamp_col_name].values

        start_timestamp_shimmer = time_stamps_shimmer[0]
        end_timestamp_shimmer =  time_stamps_shimmer[-1]

        start_time = dt.datetime.fromtimestamp(start_timestamp_shimmer, tz)
        end_time  = dt.datetime.fromtimestamp(end_timestamp_shimmer, tz)

        print("shimmer_baseline start time: "+str(start_time))
        print("shimmer_baseline stop time: "+str(end_time))
    
        ####------------------------------------------------------------------

        start_timestamp_task= task_onset_float_timestamp[ind_timestamp]
        stop_timestamp_task = get_offset_timestamp(start_timestamp_task, offset_min)
        sliced_ecg_df_task=slice_df_wrt_timestamps(selected_ecg_df, start_timestamp_task, stop_timestamp_task, shimmer_timestamp_col_name)
        sliced_ecg_df_task=sliced_ecg_df_task.reset_index(drop=True)
        ###-------------------------------------------------
        ecg_array_task = sliced_ecg_df_task["ecg"].values
        out_task = ecg.ecg(signal=ecg_array_task, sampling_rate=sampling_freq_shimmer, show=False, interactive=False)
        sliced_ecg_df_task["filtered"] = out_task["filtered"]
        r_peaks_ind_task = out_task["rpeaks"]
        rr_msec_task=from_rr_ind_2_msec(r_peaks_ind_task)
        rr_msec_task_clean=remove_outliers_rri(rr_msec_task, sampling_freq = 256, outlier_std = 1.5)
        
        time_domain_features=timedomain(rr_msec_task_clean)
        ###--------------------------------------------------
        mean_hbpm_task = time_domain_features["Mean HR (beats/min)"]
        std_hbpm_task = time_domain_features["STD HR (beats/min)"]
        
        all_ibi_one_list_one_task.extend(rr_msec_task_clean)
        all_ibi_sublist_one_task.append(rr_msec_task_clean)
        
        mean_cumm_list.append(np.mean(rr_msec_task_clean))
        std_cumm_list.append(np.std(rr_msec_task_clean))
        
        mean_hbpm_cumm_list.append(mean_hbpm_task)
        std_hbpm_cumm_list.append(std_hbpm_task)
        
        #print("TASK DATA")
        #fig_name = "Task"+ sub_id + ".png"
        #fig_name = "Task"+str(ind_timestamp)+".png"
        #plt_title_raw = "Raw ECG: Task"
        #plt_title_filtered = "Filtered ECG: Task"
        #plot_ecg_wf(sliced_ecg_df_task,r_peaks_ind_task,offset_min, dest_path_save_figures,fig_name, 
         #           nr_plots=total_plots, fig_title_raw=plt_title_raw, fig_title_filtered=plt_title_filtered)
    
        #####------------------------------------------------------------
        ##### -------------for debugging---------------------------------
        #####------------------------------------------------------------

        start_time = dt.datetime.fromtimestamp(start_timestamp_task, tz)
        end_time  = dt.datetime.fromtimestamp(stop_timestamp_task, tz)

        print("task start time: "+str(start_time))
        print("task stop time: "+str(end_time))

        time_stamps_shimmer= sliced_ecg_df_task[shimmer_timestamp_col_name].values

        start_timestamp_shimmer = time_stamps_shimmer[0]
        end_timestamp_shimmer =  time_stamps_shimmer[-1]

        start_time = dt.datetime.fromtimestamp(start_timestamp_shimmer, tz)
        end_time  = dt.datetime.fromtimestamp(end_timestamp_shimmer, tz)

        print("shimmer_task start time: "+str(start_time))
        print("shimmer_task stop time: "+str(end_time))

        ####-------------------------------------------------------------
        ####-------------------------------------------------------------
        ####-------------------------------------------------------------
    
        start_timestamp_recovery= recovery_onset_list_timestamps[ind_timestamp]
        stop_timestamp_recovery =get_offset_timestamp(start_timestamp_recovery, offset_min)
        sliced_ecg_df_recovery=slice_df_wrt_timestamps(selected_ecg_df, start_timestamp_recovery, stop_timestamp_recovery, shimmer_timestamp_col_name)
        sliced_ecg_df_recovery=sliced_ecg_df_recovery.reset_index(drop=True)

        ###-------------------------------------------------
        ecg_array_recovery = sliced_ecg_df_recovery["ecg"].values
        out_recovery = ecg.ecg(signal=ecg_array_recovery, sampling_rate=sampling_freq_shimmer, show=False, interactive=False)
        sliced_ecg_df_recovery["filtered"] = out_recovery["filtered"]
        r_peaks_ind_recovery = out_recovery["rpeaks"]
        rr_msec_recovery=from_rr_ind_2_msec(r_peaks_ind_recovery)
        rr_msec_recovery_clean=remove_outliers_rri(rr_msec_recovery, sampling_freq = 256, outlier_std = 1.5)
        time_domain_features=timedomain(rr_msec_recovery_clean)
        ###--------------------------------------------------
        mean_hbpm_recovery = time_domain_features["Mean HR (beats/min)"]
        std_hbpm_recovery = time_domain_features["STD HR (beats/min)"]
        ###-----------------------------------------------------
        all_ibi_one_list_one_task.extend(rr_msec_task_clean)
        all_ibi_sublist_one_task.append(rr_msec_task_clean)
        mean_cumm_list.append(np.mean(rr_msec_recovery_clean))
        std_cumm_list.append(np.std(rr_msec_recovery_clean))
        
        mean_hbpm_cumm_list.append(mean_hbpm_recovery)
        std_hbpm_cumm_list.append(std_hbpm_recovery)
        
        
        #####------------------------------------------------------------
        ##### -------------for debugging---------------------------------
        #####------------------------------------------------------------
        start_time = dt.datetime.fromtimestamp(start_timestamp_recovery, tz)
        end_time  = dt.datetime.fromtimestamp(stop_timestamp_recovery, tz)

        print("recovery start time: "+str(start_time))
        print("recovery stop time: "+str(end_time))


        time_stamps_shimmer= sliced_ecg_df_recovery[shimmer_timestamp_col_name].values

        start_timestamp_shimmer = time_stamps_shimmer[0]
        end_timestamp_shimmer =  time_stamps_shimmer[-1]

        start_time = dt.datetime.fromtimestamp(start_timestamp_shimmer, tz)
        end_time  = dt.datetime.fromtimestamp(end_timestamp_shimmer, tz)

        print("shimmer_recovery start time: "+str(start_time))
        print("shimmer_recovery stop time: "+str(end_time))

        ####-------------------------------------------------------------
        ####-------------------------------------------------------------
        ####-------------------------------------------------------------
        
        
        hrv_data_dict = {}
        
        hrv_data_dict["subject_id"] = file_name_ii
        
        event_interets_plot = ["Baseline", "Task", "Recovery"]
    
        hrv_data_dict["event_interst"] = event_interets_plot
    
        hrv_data_dict["all_rr_one_list"] = all_ibi_one_list_one_task
    
        hrv_data_dict["all_rr_sublists"] = all_ibi_sublist_one_task
        
        hrv_data_dict["mean_hbpm"] = mean_hbpm_cumm_list
        hrv_data_dict["std_hbpmy"] = std_hbpm_cumm_list
        
        hrv_data_dict["mean_ibi"] = mean_cumm_list
        hrv_data_dict["std_ibi"] = std_cumm_list
        
        hrv_data_dict["task_type"] = task_type_ii
        
        
        pkl_file_hrv = file_name_trig[:-4] + "_"+task_type_ii+ ".pkl"
        
        print(pkl_file_hrv)
        
        dest_path_pkl_file = os.path.join(dest_path, pkl_file_hrv)
        with open(dest_path_pkl_file, "wb") as f:
            pickle.dump(hrv_data_dict, f)
            f.close()
            
        title_fig =  task_type_ii+ ":" + file_name_trig[:-4]
        fig_name =  file_name_trig[:-4]+ "_" + task_type_ii+".png"
        fig_dest_path = os.path.join(dest_path,fig_name)
        y_label_fig = "HBPM"
        fig_mean_std= plot_mean_std(mean_hbpm_cumm_list, std_hbpm_cumm_list, event_interets_plot, title_fig, y_label_fig)
        
        fig_mean_std.savefig(fig_dest_path)
        
        
        
        #mean_norm_list, std_norm_list = min_max_norm(all_ibi_one_list_one_task, all_ibi_sublist_one_task)
        
        #title_fig =  file_name_ii[:-4] + task_type_ii 
        #y_label_fig = "Norm-IBI"
        #fig_norm_mean_std= plot_mean_std(mean_norm_list, std_norm_list, event_interets_plot, title_fig, y_label_fig)
        
        
        
    
        ###-------------------------------------------------------------------
        ####-------------------for raw and filtered plotting-------------------
        ####-------------------------------------------------------------------
        #print("RECOVERY DATA")
        #fig_name="Recovery_"+ sub_id + ".png"
        #fig_name = "Recovery"+str(ind_timestamp)+".png"
        #plt_title_raw = "Raw ECG: Recovery"
        #plt_title_filtered = "Filtered ECG: Recovery"
        #plot_ecg_wf(sliced_ecg_df_recovery,r_peaks_ind_recovery,offset_min, dest_path_save_figures, 
         #           fig_name, nr_plots=total_plots, fig_title_raw=plt_title_raw, fig_title_filtered=plt_title_filtered )

        ####-------------------------------------------------------------------------
    
    
    
    
    
    

        